In [19]:
import pandas as pd
import psycopg2
from psycopg2 import sql

## Database Querying

In [37]:

# Function to connect to the PostgreSQL database
def connect_to_db():
    return psycopg2.connect(
        dbname="ny_taxi",
        user="root",
        password="root",
        host="localhost",
        port="5432"
    )

# Function to execute query and return results as DataFrame
def query_db(query, conn, params=None):
    with conn.cursor() as cursor:
        cursor.execute(query, params)
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
        return pd.DataFrame(data, columns=columns)

# Establish connection to database
conn = connect_to_db()

# Queries
# 1) How many taxi trips were totally made on September 18th 2019?
query1 = """
SELECT COUNT(*) AS trip_count
FROM taxi_table
WHERE lpep_pickup_datetime >= '2019-09-18 00:00:00'
AND lpep_dropoff_datetime < '2019-09-19 00:00:00';
"""

# 2) Which was the pick up day with the largest trip distance?
query2 = """
SELECT lpep_pickup_datetime::date AS pickup_day, SUM(trip_distance) AS total_distance
FROM taxi_table
WHERE lpep_pickup_datetime::date IN ('2019-09-18', '2019-09-16', '2019-09-26', '2019-09-21')
GROUP BY pickup_day
ORDER BY total_distance DESC
LIMIT 1;
"""

# 3) The 3 pick up Boroughs on '2019-09-18' with a sum of total_amount superior to 50000
query3 = """
SELECT l."Borough", SUM(t."total_amount") AS total_amount
FROM "taxi_table" t
JOIN "taxi_lookup" l ON t."PULocationID" = l."LocationID"
WHERE t."lpep_pickup_datetime"::date = '2019-09-18'
AND l."Borough" != 'Unknown'
GROUP BY l."Borough"
HAVING SUM(t."total_amount") > 50000
ORDER BY total_amount DESC
LIMIT 3;
"""

# 4) For the passengers picked up in Astoria, the drop off zone with the largest tip in September 2019
query4 = """
SELECT l2."Zone" AS dropoff_zone, MAX(t."tip_amount") AS max_tip
FROM "taxi_table" t
JOIN "taxi_lookup" l1 ON t."PULocationID" = l1."LocationID"
JOIN "taxi_lookup" l2 ON t."DOLocationID" = l2."LocationID"
WHERE l1."Zone" = 'Astoria'
AND t."lpep_pickup_datetime" >= '2019-09-01'
AND t."lpep_pickup_datetime" < '2019-10-01'
GROUP BY l2."Zone"
ORDER BY max_tip DESC
LIMIT 1;
"""

# Execute queries
df1 = query_db(query1, conn)
df2 = query_db(query2, conn)
df3 = query_db(query3, conn)
df4 = query_db(query4, conn)

# Close the database connection
conn.close()

def display_df(df, label):
    print(f"------- {label} -------")
    print(df)
    print("\n")

display_df(df1, "Trip Count on September 18th 2019")
display_df(df2, "Pickup Day with Largest Trip Distance")
display_df(df3, "Top 3 Boroughs on September 18th 2019 by Total Amount")
display_df(df4, "Largest Tip for Drop Off Zone from Astoria in September 2019")

------- Trip Count on September 18th 2019 -------
   trip_count
0       15612


------- Pickup Day with Largest Trip Distance -------
   pickup_day  total_distance
0  2019-09-26        58759.94


------- Top 3 Boroughs on September 18th 2019 by Total Amount -------
     Borough  total_amount
0   Brooklyn      96333.24
1  Manhattan      92271.30
2     Queens      78671.71


------- Largest Tip for Drop Off Zone from Astoria in September 2019 -------
  dropoff_zone  max_tip
0  JFK Airport    62.31




## Dataframe Operations

In [33]:
# 1) How many taxi trips were totally made on September 18th 2019?

df = pd.read_csv('green_tripdata_2019-09.csv',low_memory=False)

# Convert the 'lpep_pickup_datetime' column to datetime
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])

# List of dates to check
dates_to_check = ['2019-09-18', '2019-09-16', '2019-09-26', '2019-09-21']

# Initialize a dictionary to store the maximum trip distance for each date
largest_trip_each_day = {}


for date in dates_to_check:
    # Filter the dataframe for the specific date and calculate the maximum trip distance
    max_trip_distance = df[df['lpep_pickup_datetime'].dt.date == pd.to_datetime(date).date()]['trip_distance'].max()
    largest_trip_each_day[date] = max_trip_distance


largest_trip_each_day

{'2019-09-18': 70.28,
 '2019-09-16': 114.3,
 '2019-09-26': 341.64,
 '2019-09-21': 135.53}

In [34]:
# 2) Which was the pick up day with the largest trip distance?

df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

# Filter the DataFrame for trips that started and ended on September 18th, 2019
trips_on_sept_18 = df[(df['lpep_pickup_datetime'].dt.date == pd.to_datetime('2019-09-18').date()) &
                      (df['lpep_dropoff_datetime'].dt.date == pd.to_datetime('2019-09-18').date())]

# Calculate the number of trips
num_trips_on_sept_18 = len(trips_on_sept_18)

num_trips_on_sept_18

15612

In [35]:
# 3) The 3 pick up Boroughs on '2019-09-18' with a sum of total_amount superior to 50000

df = pd.read_csv('green_tripdata_2019-09.csv',low_memory=False)
lookup = pd.read_csv('taxi+_zone_lookup.csv')  # The path to your lookup table CSV


# Convert 'lpep_pickup_datetime' to datetime
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])

# Filter the DataFrame for trips on September 18th, 2019
df_sept_18 = df[df['lpep_pickup_datetime'].dt.date == pd.to_datetime('2019-09-18').date()]

# Merge the trip data with the lookup table to associate each trip with a borough
df_sept_18_with_boroughs = df_sept_18.merge(lookup, how='left', left_on='PULocationID', right_on='LocationID')

# Filter out unknown boroughs
df_known_boroughs = df_sept_18_with_boroughs[df_sept_18_with_boroughs['Borough'] != 'Unknown']

# Group by borough and sum the total_amount, then filter for sums greater than 50000
borough_totals = df_known_boroughs.groupby('Borough')['total_amount'].sum().reset_index()
boroughs_over_50k = borough_totals[borough_totals['total_amount'] > 50000]

# Find the top 3 boroughs with the highest total_amount
top_3_boroughs = boroughs_over_50k.nlargest(3, 'total_amount')

top_3_boroughs


,Borough,total_amount
1,Brooklyn,96333.24
2,Manhattan,92271.30
3,Queens,78671.71


In [36]:
# 4) For the passengers picked up in Astoria, the drop off zone with the largest tip in September 2019

df = pd.read_csv('green_tripdata_2019-09.csv',low_memory=False)
zone_lookup = pd.read_csv('taxi+_zone_lookup.csv')  # The path to your lookup table CSV

# Ensure datetime format is correct
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])

# Filter the trips for September 2019
df_september = df[df['lpep_pickup_datetime'].dt.month == 9]

# Merge with the zone lookup table to get the pickup zone names
df_september = df_september.merge(zone_lookup, left_on='PULocationID', right_on='LocationID', suffixes=('', '_pickup'))

# Filter for trips that were picked up in 'Astoria'
astoria_trips = df_september[df_september['Zone'] == 'Astoria']

# Find the record with the largest tip amount
max_tip_record = astoria_trips.loc[astoria_trips['tip_amount'].idxmax()]

# Get the drop-off location ID from the record with the largest tip
max_tip_dropoff_location_id = max_tip_record['DOLocationID']

# Lookup the zone name for the drop-off location
max_tip_dropoff_zone = zone_lookup[zone_lookup['LocationID'] == max_tip_dropoff_location_id]['Zone'].values[0]

max_tip_dropoff_zone


'JFK Airport'

------- Trip Count on September 18th 2019 -------
   trip_count
0       15612


------- Pickup Day with Largest Trip Distance -------
   pickup_day  total_distance
0  2019-09-26        58759.94


------- Top 3 Boroughs on September 18th 2019 by Total Amount -------
     Borough  total_amount
0   Brooklyn      96333.24
1  Manhattan      92271.30
2     Queens      78671.71


------- Largest Tip for Drop Off Zone from Astoria in September 2019 -------
  dropoff_zone  max_tip
0  JFK Airport    62.31


